# Simple chat-bot example

In [11]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

In [15]:
patch_otvety = r'D:\train\Otvety/Otvety.txt'
patch_prepared_answers = r'D:\train\Otvety/prepared_answers.txt'

In [22]:
#Small preprocess of the answers

question = None
written = False

with open(patch_prepared_answers, "w",encoding = 'utf-8') as fout:
    with open(patch_otvety, "r",encoding = 'utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line =='':
                continue        
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<ipython-input-22-c3a80167b521>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [30]:

data = ''
with open(patch_prepared_answers, "w",encoding = 'utf-8') as fout:
    with open(patch_otvety, "r",encoding = 'utf-8') as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                if data !="":
                    fout.write(data.strip()+'\n')
                data = ''
                continue
            my_line = line.replace("\t", " ")
            my_line = my_line.replace("\n", " ").strip()
            data += my_line + "\t"

<ipython-input-30-889096ef2c1f>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [39]:
def preprocess_txt(line,exclude,morpher,sw):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [44]:

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(patch_otvety, "r",encoding='utf-8') as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line,exclude,morpher,sw)
        sentences.append(spls)
sentences = [i for i in sentences if len(i) > 2]

<ipython-input-44-22c84a030499>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


In [59]:
patch_sentences = r'D:\train\Otvety/sentences.txt'
patch_w2w = r'D:\train\Otvety/w2v.bin'
patch_ft = r'D:\train\Otvety/ft.bin'

In [58]:
with open(patch_sentences, 'w',encoding='utf-8') as f:
    for s in sentences:
        f.write(str(s) + '\n')



In [ ]:
# with open(patch_sentences, 'r') as f:
#     sentences = [line.rstrip('\n') for line in f]

In [75]:
%%time
modelW2V = Word2Vec(sentences=sentences,  window=5, min_count=10,vector_size=300)

Wall time: 6min 57s


In [76]:
modelW2V.wv.save_word2vec_format(patch_w2w)

In [77]:
%%time
modelFT = FastText(sentences=sentences,  min_count=10, window=5, workers=14,vector_size=300)

Wall time: 18min 41s


In [78]:
modelFT.wv.save_word2vec_format(patch_ft)

In [90]:
%%time
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

with open(patch_prepared_answers, "r",encoding='utf-8') as f:
    for line in tqdm_notebook(f):
        n_w2v = 0
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1:]
        question = preprocess_txt(spls[0],exclude,morpher,sw)
        
        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)
            
        counter += 1
        

w2v_index.build(10)
ft_index.build(10)

<timed exec>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Wall time: 1h 15min 41s


True

In [93]:
patch_w2v_index = r'D:\train\Otvety/w2v_index_full'
patch_ft_index = r'D:\train\Otvety/ft_index+full'
patch_index_map = r'D:\train\Otvety/index_map.pkl'

In [94]:
w2v_index.save(patch_w2v_index)
ft_index.save(patch_ft_index)


True

In [95]:
with open(patch_index_map, 'wb') as f:
    pickle.dump(index_map, f)


In [112]:
def get_respons(question, index, model, index_map):
    question = preprocess_txt(question,exclude,morpher,sw)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 1)
    return np.random.choice(index_map[answers[0]])

In [113]:
def get_responses(question, index, model, index_map):
    question = preprocess_txt(question,exclude,morpher,sw)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model:
            vector += model[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3)
    return [index_map[i][:3] for i in answers]

In [120]:
TEXT = "как научиться читать мысли?"

In [121]:
get_respons(TEXT, w2v_index, modelW2V.wv, index_map)

'У меня бы не хватило терпения их все потом записывать! )).'

In [122]:
get_responses(TEXT, w2v_index, modelW2V.wv, index_map)

[['и жить на помойке?...нет.', 'А я уже умею!!!!.', 'ага)) о4енб).'],
 ['Примерно также, как вы видите мысли собственные. У них общая природа, почему же их можно воспринимать иначе? Если интересуют книги, то пожалуй, Звенящие кедры России Владимира Мегре..',
  'видят сердцем.',
  'Они их слышат на ощюпь....))))).'],
 ['Я бы не хотел что бы мои мысли читали. Итак очень трудно уединиться, а так мы не можем спокойно подумать, побыть наедине. Это было бы ужас!!! !<br>А вот чужие мысли я бы почитать не прочь, но только по моему желанию, а то надоест так =))).',
  'не знаю, мне и безкомного общения хватает, а мысли чужие меня мало интересуют, тем более, многое и так на лицах написано.',
  'Не думаю, т. к. довольно сложно контролировать свои собственные мысли..']]

In [123]:
get_respons(TEXT, ft_index, modelFT.wv, index_map)

'У меня бы не хватило терпения их все потом записывать! )).'

In [124]:
get_responses(TEXT, w2v_index, modelFT.wv, index_map)

[['С днём рожденья тебя поздравляю!<br>Здоровья, радости желаю.<br>И чтоб жила ты много лет<br>Без слёз, без горести, без бед!<br><br>2<br><br>Желаю радости всегда<br>И настроенья бодрого,<br>Не знать печали никогда<br>И в жизни всего доброго.<br>Никогда не унывать,<br>Не видеть огорчения,<br>И дни с улыбкой начинать,<br>Как в этот День Рождения!<br><br>3<br><br>С днём рожденья тебя поздравляю.<br>Веселись в этот день, не грусти.<br>Много счастья тебе я желаю,<br>А особенно счастья в любви!<br><br>4<br><br>Желаю счастья целый ворох,<br>Улыбок радостных букет,<br>Друзей надёжных и весёлых,<br>Счастливой жизни целый век!<br><br>5<br><br>Желаю тебе я<br>В твой день рождения<br>Здоровья, успехов,<br>Удач и везенья,<br>Лёгких дорог,<br>Возвращений счастливых,<br>Мой самый желанный<br>И самый красивый!<br><br>6<br><br>Будь девушкой, уверенной в себе,<br>В своей борьбе, в тропе своей, в судьбе.<br>В толпе держись, как будто ты одна,<br>Как будто ты Богиней рождена!<br><br>7<br><br>Если ты про

In [125]:
TEXT = "Сегодня будет секс?"

In [127]:
get_respons(TEXT, ft_index, modelFT.wv, index_map)

'Присылай сисьги я подумаю над твоим предложением ))).'

In [129]:
get_responses(TEXT, ft_index, modelFT.wv, index_map)

[['тот от которого крышу сносит?)).',
  'нет.',
  'Обязательно! И не один раз! И разнообразный)).'],
 ['Кто-нибудь ХОЧЕТ?Я ХОЧУ...Здесь кто-нибудь ХОЧЕТ?Я ХОЧУ...Отлично и еще ВОПРОС...Кто-нибудь ХОЧЕТ?Я ХОЧУ...Здесь кто-нибудь ХОЧЕТ?Я ХОЧУ....',
  'ты предлагаешь создать групу ХОТЕЛОК ? :)).',
  'все хотят.. . я хочу....'],
 ['Я! Но муж на работе=( Мот кто поможет, а зай? =).',
  'я хочу! хочу и не молчу).',
  'нас двое. предложи женщину. здесь все хотят.']]